In [152]:
from vectorizer import cluster_paragraphs
from vectorizer import vectorize_text
from random import shuffle
import pandas as pd
from nltk.corpus import stopwords
from collections import Counter
import nltk



In [148]:
# Separando dados dos países a serem avaliados
df = pd.read_csv('/home/raime/Documentos/Repositórios/text_clustering/latam.csv')
nomes_colunas = df.columns
dados_brasil = df.loc[df[' País donde se chequeó la desinformación']== "Brasil"]
dados_mexico = df.loc[df[' País donde se chequeó la desinformación']== "México"]

In [150]:
# criação de 2 listas de notícias, uma para o brasil e outra para o mexico
 
noticias_brasil = dados_brasil['Desinformación o afirmación chequeada (o título del explicador)']

lista_de_noticias_br = []

for noticia in noticias_brasil:
    lista_de_noticias_br.append(noticia)

type(lista_de_noticias_br)
# lista de noticias_br extraídas do banco de dados.

noticias_mexico = dados_mexico['Desinformación o afirmación chequeada (o título del explicador)']

lista_de_noticias_me = []

for news in noticias_mexico:
    lista_de_noticias_me.append(news)

type(lista_de_noticias_me)

#lista_de_noticias_me extraídas do banco de dados.

list

In [ ]:
# vetorizando as noticias do brasil e do mexico
vetores_br = []
for i in lista_de_noticias_br:
    vetores_br.append(vectorize_text(i))

vetores_me = []
for t in lista_de_noticias_me:
    vetores_me.append(vectorize_text(t))

vetores_me

In [ ]:
# preparo de dados para contagem de palavras mais comuns
bag_of_words_br = []
for vetores in vetores_br:
    for palavras in vetores:
        bag_of_words_br.append(palavras)

bag_of_words_me = []
for vectores in vetores_me:
    for palabras in vectores:
        bag_of_words_me.append(palabras)

bag_of_words_br

In [ ]:
# stopwords em espanhol: fonte nltk.com
from nltk.corpus import stopwords
spanish_stopwords = stopwords.words('spanish')

print(spanish_stopwords)

In [160]:
# removendo stopwords
stopword = nltk.corpus.stopwords.words('spanish')

def remove_stopwords(bag_of_words_br):
    bag_of_words_br = [word for word in bag_of_words_br if word not in stopword]
    return bag_of_words_br

bag_of_words_clean_br = remove_stopwords(bag_of_words_br) # brasil

def remove_stoppalabras(bag_of_words_me):
    bag_of_words_me = [word for word in bag_of_words_me if word not in stopword]
    return bag_of_words_me

bag_of_words_clean_me = remove_stoppalabras(bag_of_words_me) # mexico

In [187]:
# contagem das palavras mais comuuns com contagem
from collections import Counter
b = Counter(bag_of_words_clean_br)
palavras_comuns_br = b.most_common(50)

m = Counter(bag_of_words_clean_me)
palavras_comuns_me = m.most_common(50)

In [ ]:
# salvando tabela de palavras mais comuuns do brasil e do mexico
brasil_tabela_palavras_comuns = pd.DataFrame((palavras_comuns_br))
palavras_brasileiras = brasil_tabela_palavras_comuns.rename(columns={0:'Palavras mais comuns', 1:'Quantas vezes apareceu'})
palavras_brasileiras.to_excel('PalavrasBrasileirasComuns.xlsx')

mexico_tabela_palavras_comuns = pd.DataFrame((palavras_comuns_me))
palavras_mexicanas = mexico_tabela_palavras_comuns.rename(columns={0:'Palavras mais comuns', 1:'Quantas vezes apareceu'})
palavras_mexicanas.to_excel('PalavrasMexicanasComuns.xlsx')

In [ ]:
# paramos na vetorização das notícias